In [1]:
from transformers import AutoTokenizer, MBartForConditionalGeneration
from peft import PeftModel
import torch

# 1. Load the correct base model
base_model = MBartForConditionalGeneration.from_pretrained("CAMeL-Lab/arabart-qalb15-gec-ged-13")
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/arabart-qalb15-gec-ged-13")

# 2. Load fine-tuned LoRA adapter
model = PeftModel.from_pretrained(base_model, "fine_tuned_model")

# 3. Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

c:\Users\mizog\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at CAMeL-Lab/arabart-qalb15-gec-ged-13 were not used when initializing MBartForConditionalGeneration: ['model.encoder.embed_ged_tags.weight']
- This IS expected if you are initializing MBartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PeftModel(
  (base_model): LoraModel(
    (model): MBartForConditionalGeneration(
      (model): MBartModel(
        (shared): MBartScaledWordEmbedding(50002, 768, padding_idx=1)
        (encoder): MBartEncoder(
          (embed_tokens): MBartScaledWordEmbedding(50002, 768, padding_idx=1)
          (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
          (layers): ModuleList(
            (0-5): 6 x MBartEncoderLayer(
              (self_attn): MBartSdpaAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
   

In [3]:
# Example sentences with spelling/grammar issues
sentences = [
   "الشعاب العربي",
    "اناا بحب المشا كثير.",
    "المدرس قال لنا ان نذاكر جيدا.",
    "هاذي البنت ذكيه جدا.",
    "ذهبت الى الجامعه لاكنها كانت مغلقه.",
    "شوفت احمد في السوق يوم الخميس.",
    "الطفل يلعب في الشارع لوحده.",
    "الصديقاتهم ذهبوا إلى الحديقه.",
    "اكلت فطور الصباح بسرعه لانني كنت جعانه.",
    "لازم تروحو للبيت الحين.",
    "هذاك الولد ما يبي يدرس.",
    "امس الجو كان ممطره.",
    "المدرسه تقع جنب بيتناا.",
    "ابي راح لمكه لأداء العمره."
]

# Run the model on each sentence
for i, sentence in enumerate(sentences, 1):
    # Tokenize and move input to the right device
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    
    # Generate corrected output
    outputs = model.generate(**inputs, max_length=128)
    
    # Decode the corrected sentence
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"{i}. Original : {sentence}")
    print(f"   Corrected: {corrected}")
    print("-" * 50)


1. Original : الشعاب العربي
   Corrected: التاريخ العربي
--------------------------------------------------
2. Original : اناا بحب المشا كثير.
   Corrected: أنا أحب اللعب كثيرا
--------------------------------------------------
3. Original : المدرس قال لنا ان نذاكر جيدا.
   Corrected: المدرس قال لنا ان نذاكر جيدا
--------------------------------------------------
4. Original : هاذي البنت ذكيه جدا.
   Corrected: هذه البنت ذكيه جدا
--------------------------------------------------
5. Original : ذهبت الى الجامعه لاكنها كانت مغلقه.
   Corrected: ذهبت الى الجامعة لكنها كانت مغلقة
--------------------------------------------------
6. Original : شوفت احمد في السوق يوم الخميس.
   Corrected: شاهدت أحمد في السوق يوم الخميس
--------------------------------------------------
7. Original : الطفل يلعب في الشارع لوحده.
   Corrected: الطفل يلعب في الشارع لوحده
--------------------------------------------------
8. Original : الصديقاتهم ذهبوا إلى الحديقه.
   Corrected: الصديقات ذهبوا إلى الحدود
-------